# Try to do regression using GPT-3

In [2]:
import time

from fastcore.utils import save_pickle
from sklearn.model_selection import train_test_split

from gpt3forchem.api_wrappers import extract_prediction, fine_tune, query_gpt3
from gpt3forchem.data import get_polymer_data
from gpt3forchem.input import create_single_property_forward_prompts

In [3]:
df = get_polymer_data()
df_train, df_test = train_test_split(df, train_size=200, random_state=42)


In [4]:
train_prompts = create_single_property_forward_prompts(
    df_train, "deltaGmin_cat", {"deltaGmin_cat": "adsorption energy"}, encode_value=False
)

test_prompts = create_single_property_forward_prompts(
    df_test, "deltaGmin_cat", {"deltaGmin_cat": "adsorption energy"}, encode_value=False
)


In [5]:
train_prompts

,prompt,completion
0,what is the adsorption energy of B-R-B-B-R-R-R...,medium@@@
1,what is the adsorption energy of R-A-R-B-R-W-W...,large@@@
2,what is the adsorption energy of W-R-W-R-B-A-W...,small@@@
3,what is the adsorption energy of W-R-W-B-W-A-A...,large@@@
4,what is the adsorption energy of A-A-W-R-R-B-R...,large@@@
...,...,...
195,what is the adsorption energy of R-R-B-B-W-R-W...,very small@@@
196,what is the adsorption energy of A-B-A-W-W-B-W...,medium@@@
197,what is the adsorption energy of R-W-R-B-B-R-B...,very large@@@
198,what is the adsorption energy of R-W-W-A-W-R-B...,large@@@
